仅通过强化学习，只能得到一个在特定场景、特定对象、特定任务、特定具身构型的策略，泛化性很差。故而需要探寻新的学习范式和网络架构

# Imitation Learning

## 理论铺垫


### 1. 模仿学习的基本框架

#### 1.1 问题定义

在强化学习框架下，智能体与环境的交互可以建模为马尔可夫决策过程 (Markov Decision Process, MDP)：

$$
\mathcal{M} = (\mathcal{S}, \mathcal{A}, \mathcal{P}, \mathcal{R}, \gamma)
$$

其中：
- $\mathcal{S}$：状态空间
- $\mathcal{A}$：动作空间
- $\mathcal{P}(s'|s,a)$：状态转移概率
- $\mathcal{R}(s,a)$：奖励函数
- $\gamma \in [0,1)$：折扣因子

**模仿学习 (Imitation Learning, IL)** 的目标是：给定专家策略 $\pi_E$ 的演示数据，学习一个学生策略 $\pi_\theta$，使其行为尽可能接近专家。

与传统强化学习的区别：
- **RL**：通过与环境交互，最大化累积奖励 $\mathbb{E}\left[\sum_{t=0}^\infty \gamma^t r_t\right]$
- **IL**：通过模仿专家，最小化与专家行为的差异

---

### 2. Behavior Cloning (BC)

#### 2.1 核心思想

**Behavior Cloning** 是最直接的模仿学习方法，它将策略学习问题转化为**监督学习问题**。

> **核心假设**：如果学生策略在每个状态下都能准确预测专家的动作，那么它就能复现专家的行为。

#### 2.2 数学形式

给定专家演示数据集：
$$
\mathcal{D}_E = \{(s_i, a_i^E)\}_{i=1}^N
$$

其中 $a_i^E = \pi_E(s_i)$ 是专家在状态 $s_i$ 下的动作。

BC 的目标是最小化学生策略与专家动作之间的差异：
$$
\min_\theta \mathcal{L}_{BC}(\theta) = \mathbb{E}_{(s,a^E) \sim \mathcal{D}_E} \left[ \| \pi_\theta(s) - a^E \|^2 \right]
$$

对于**连续动作空间**（如你的 $se(3)$ 旋量），通常使用均方误差 (MSE)；对于**离散动作空间**，使用交叉熵损失。

#### 2.3 优势

1. **实现简单**：标准的监督学习流程，易于调试和复现。
2. **样本效率高**：不需要与环境交互，可以离线训练。
3. **稳定性好**：训练过程可控，不会出现 RL 中的性能崩溃。

#### 2.4 核心问题：分布偏移 (Distribution Shift)

BC 的最大问题是**复合误差 (Compounding Error)**，由 Ross et al. (2011) 在 DAgger 论文中首次系统分析。

##### 问题根源

**训练分布** vs **测试分布**不匹配：
- **训练时**：学生只见过专家访问的状态 \[s \sim d^{\pi_E}\]
- **测试时**：学生自己执行动作，可能偏离专家轨迹，进入未见过的状态 \[s \sim d^{\pi_\theta}\]

##### 理论分析

假设学生在每个状态下的预测误差为 $\epsilon$（即 $\mathbb{P}(\pi_\theta(s) \neq \pi_E(s)) \leq \epsilon$），则在 $T$ 步轨迹中：

$$
\mathbb{E}[\text{Total Error}] = O(\epsilon T^2)
$$

**关键洞察**：误差是**二次增长**的！即使单步误差很小（如 $\epsilon = 0.01$），在长轨迹中也会累积成巨大的偏差。

##### 直观解释

想象一个自动驾驶场景：
1. 专家总是在车道中央行驶。
2. 学生通过 BC 学会了"在车道中央时如何驾驶"。
3. 但如果学生稍微偏离车道（进入训练数据未覆盖的状态），它不知道如何纠正，导致越偏越远。

---

### 3. Dataset Aggregation (DAgger)

#### 3.1 核心思想

**DAgger** (Ross et al., AISTATS 2011) 通过**迭代式数据收集**来解决分布偏移问题。

> **核心思想**：让学生在自己的状态分布下收集数据，并由专家提供标签，从而使训练分布逐渐接近测试分布。

#### 3.2 算法流程

**输入**：专家策略 $\pi_E$，初始数据集 $\mathcal{D}_0$  
**输出**：学生策略 $\pi_\theta$

**迭代过程**（Round $i = 1, 2, \ldots, N$）：

1. **部署学生策略**：在环境中执行 $\pi_{\theta_{i-1}}$，收集轨迹。
2. **专家标注**：对于学生访问的每个状态 $s$，查询专家动作 $a^E = \pi_E(s)$。
3. **数据聚合**：将新数据加入数据集：
   $$\mathcal{D}_i = \mathcal{D}_{i-1} \cup \{(s, a^E)\}$$
4. **重新训练**：在聚合数据集 $\mathcal{D}_i$ 上训练学生策略：
   $$\pi_{\theta_i} = \arg\min_\theta \mathbb{E}_{(s,a^E) \sim \mathcal{D}_i} \left[ \| \pi_\theta(s) - a^E \|^2 \right]$$

#### 3.3 理论保证

DAgger 提供了更强的误差界：

$$
\mathbb{E}[\text{Total Error}] = O(\epsilon T)
$$

**关键改进**：误差从**二次增长**降为**线性增长**！

##### 证明直觉

- BC：学生只在 $d^{\pi_E}$ 下训练，偏离后无法纠正 → 误差累积。
- DAgger：学生在 $d^{\pi_\theta}$ 下收集数据，学会了"如何从错误中恢复" → 误差不再累积。

#### 3.4 实践中的挑战

尽管理论优雅，DAgger 在实际应用中面临以下问题：

1. **专家查询成本高**：
   - 如果专家是人类（如遥操作），每轮迭代都需要大量人工标注。
   - 如果专家是复杂规划器（如 MPC），计算成本高。

2. **多轮迭代的工程复杂度**：
   - 需要维护多个版本的数据集。
   - 需要多次停止训练、收集数据、重新训练。

3. **安全性问题**：
   - 在真实机器人上，让学生"犯错"可能导致硬件损坏或安全风险。

**这些问题在仿真环境（如 IsaacLab）中可以得到缓解**，因为：
- 专家策略（神经网络）查询成本极低。
- 可以并行运行数千个环境实例。
- 仿真中的"失败"没有物理成本。

---

### 4. RL + DAgger：混合范式

#### 4.1 动机

纯 DAgger 只关注"模仿专家"，但没有利用环境的奖励信号。这在以下场景中是次优的：

1. **专家不完美**：专家策略可能在某些状态下表现不佳。
2. **任务目标明确**：如果我们有清晰的奖励函数（如手内旋转的位姿误差），为什么不直接优化它？
3. **泛化需求**：学生可能需要在专家未见过的场景中表现良好。

**RL + DAgger** 结合了两者的优势：
- **DAgger** 提供强有力的初始引导，避免盲目探索。
- **RL** 允许学生在专家基础上进一步优化，甚至超越专家。

#### 4.2 混合目标函数

学生策略同时优化两个目标：

$$
J(\theta) = \underbrace{\mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^T \gamma^t r_t \right]}_{J_{RL}(\theta): \text{最大化任务奖励}} - \lambda \underbrace{\mathbb{E}_{s \sim d^{\pi_\theta}} \left[ \| \pi_\theta(s) - \pi_E(s) \|^2 \right]}_{L_{Imitation}(\theta): \text{模仿专家}}
$$

其中：
- $J_{RL}$：标准的 RL 目标（如 PPO、SAC 的目标函数）。
- $L_{Imitation}$：实时查询专家的模仿损失（**关键**：在学生的状态分布 $d^{\pi_\theta}$ 下采样）。
- $\lambda$：权重系数，控制两者的平衡。

#### 4.3 梯度更新

策略参数的梯度为：

$$
\nabla_\theta J(\theta) = \nabla_\theta J_{RL}(\theta) + \lambda \nabla_\theta L_{Imitation}(\theta)
$$

**第一项（RL 梯度）**：
使用标准的策略梯度方法（如 PPO）：
$$
\nabla_\theta J_{RL} = \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^T \nabla_\theta \log \pi_\theta(a_t|s_t) A^{\pi_\theta}(s_t, a_t) \right]
$$
其中 $A^{\pi_\theta}$ 是优势函数 (Advantage Function)。

**第二项（模仿梯度）**：
标准的监督学习梯度：
$$
\nabla_\theta L_{Imitation} = \mathbb{E}_{s \sim d^{\pi_\theta}} \left[ \nabla_\theta \| \pi_\theta(s) - \pi_E(s) \|^2 \right]
$$

#### 4.4 权重衰减策略

$\lambda$ 的设置至关重要，通常采用**自适应衰减**：

$$
\lambda(t) = \lambda_0 \cdot \exp(-\alpha t) + \lambda_{min}
$$

**直觉**：
- **训练初期**（$\lambda$ 大）：学生主要模仿专家，快速学会基本技能。
- **训练后期**（$\lambda$ 小）：学生主要优化任务奖励，可能超越专家。

#### 4.5 与 DAPG 的关系

**DAPG** (Rajeswaran et al., NeurIPS 2018) 是这一思想的经典实现：

$$
\nabla_\theta J_{DAPG} = \underbrace{\mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_t \nabla_\theta \log \pi_\theta(a_t|s_t) A_t \right]}_{\text{Policy Gradient}} + \underbrace{\lambda \mathbb{E}_{(s,a) \sim \mathcal{D}_E} \left[ \nabla_\theta \log \pi_\theta(a|s) \right]}_{\text{Behavior Cloning}}
$$

**你的方法与 DAPG 的区别**：
- **DAPG**：使用**离线专家数据集** $\mathcal{D}_E$（固定分布 $d^{\pi_E}$）。
- **你的方法**：使用**在线专家查询**（动态分布 $d^{\pi_\theta}$）。

因此，你的方法可以称为 **"DAgger-Augmented Policy Gradient"** 或 **"Online DAPG"**，理论上更强（因为解决了分布偏移）。

---

### 5. 理论对比总结

| 方法 | 训练分布 | 误差界 | 优势 | 劣势 |
|:---|:---|:---|:---|:---|
| **BC** | $d^{\pi_E}$ (专家分布) | $O(\epsilon T^2)$ | 简单、稳定、样本高效 | 分布偏移严重 |
| **DAgger** | $d^{\pi_\theta}$ (学生分布) | $O(\epsilon T)$ | 解决分布偏移 | 需要多轮迭代 |
| **RL** | $d^{\pi_\theta}$ | 依赖于奖励设计 | 可以超越专家 | 探索困难、训练不稳定 |
| **RL + DAgger** | $d^{\pi_\theta}$ | $O(\epsilon T)$ + RL 保证 | 结合两者优势 | 超参数调优复杂 |

## 跨空间模仿学习
灵巧手直接在se3空间上训练，效果不佳，始终不如在Joint Space上训练的效果好。
但se3空间有着潜在的高级语义表达能力，理论上跨手型迁移效果会更好。为此采取师生策略来将Joint Space上的知识迁移到se3空间上。
<br> 目前有几个常用的算法：行为克隆（BC Behavior），DAgger，Expert Regularization，DAPG(Demonstration Augmented Policy Gradient)， Residual RL，Diffusion Policy
<br> 从社区使用情况来看，BC和RL+Expert Regularization使用的比较多。根据离线、在线等方式，有很多不同的变体。

## 目前的构想
1. 先用关节空间策略生产数据，BC预训练se3动作模型，然后观察效果
2. 如果效果不好，再在该模型基础上进行 RL + DAgger
> 但先前未涉足过模仿学习，可以先了解一下 IsaacLab 是否有相关例程

## 设计
### Bebavior cloning
关节空间和se3空间的观察和动作并不完全对齐，需要进行一些转换。这样才能进行模仿学习。
<br>关节空间主要在配置文件 `AnyRotate/source/leaphand/leaphand/tasks/manager_based/leaphand/inhand_base_env_cfg.py`
<br>se3空间主要在配置文件 `AnyRotate/source/leaphand/leaphand/tasks/manager_based/leaphand/inhand_se3_env_cfg.py`
<br>**注意**：这里的关节空间策略是相对位置增量的关节动作，se3策略是dls旋量动作

### 观察对齐
设关节空间的观察为 $O_J$, se3任务空间的观察为 $O_{T}$，则我在 Recorder 里需要记录的观察为 $O=O_J \cup O_T \cup J_b$ 。
$$\mathcal{D}_{joint}=\{O_J\}_{t-L+1}^{t}=\{\theta,p_{obj},q_{obj},X_{goal},\Delta q,a\}_{t-L+1}^{t}$$
$$\mathcal{D}_{se3}=\{O_T\}_{t-L+1}^{t}=\{\mathcal{V},p_{obj},q_{obj},X_{goal},\Delta q,a\}_{t-L+1}^{t}$$
$$\mathcal{D}_{obs}=\mathcal{D}_{joint} \cup \mathcal{D}_{se3} \cup J_b^{t-L+1:t}=\{\theta,\mathcal{V},p_{obj},q_{obj},X_{goal},\Delta q,a_{\theta},a_{\mathcal{V}},J_b\}_{t-L+1}^{t}$$
**符号说明**：
- $\theta$: Leaphand的实际关节角
- $p_{obj}$: 操作物体的质心位置
- $q_{obj}$: 操作物体的四元数姿态
- $X_{goal}$: 操作物体的目标位姿，由四元数和位置组成
- $\Delta q$: 操作物体的目标位姿相对于当前位姿的变化量
- $a_{\theta}$: 上一步的关节空间动作
- $a_{\mathcal{V}}$: 上一步的se3空间动作
- $J_b$: 刚体雅可比矩阵，参考点在{b},参考系在{b}
  
除此之外，一些固定的值也需要记录，但只需一次。如 $\lambda$, $Ad_{T_{bb'}}$

$L$ 是设置的历史步长
>注意：这里的 $\theta$ 和 $\mathcal{V}$ 并不是指原始值，而是规范化为[-1,1]的值；$a$ 是上一步动作，前者对应关节空间动作，后者对应se3空间动作。

### 动作对齐
动作对齐是模仿学习的关键，下面从 data flow 的视角看待如何将关节空间和se3空间在动作上进行对齐。这里以输出为相对关节增量的关节动作空间为例

**Joint-space policy**: $o_J \xrightarrow{\pi(\cdot)} a \xrightarrow{\text{affine}} \Delta\theta \xrightarrow{\theta_{cur}} \theta_{des} \xrightarrow{\text{clip}} \theta_{act}$

**se3-space policy**: $o_T \xrightarrow{\pi(\cdot)} a \xrightarrow{\text{affine}} \mathcal{V_{b'}} \xrightarrow{Ad_{T_{bb'}}} \mathcal{V_b} \xrightarrow{J_{dls}^+} \dot{\theta} \xrightarrow{\Delta t} \Delta\theta \xrightarrow{\text{clip}} \theta_{act}$

那么，对齐/一致的地方可在于 $\Delta\theta$，或者 $\theta_{des}$。那么，模仿监督损失可写为
$$
\min_\theta \mathcal{L}_{BC}(\theta) = \mathbb{E}_{(o,a^E) \sim \mathcal{D}_E} \left[ \| J_{dls}^+ Ad_{T_{bb'}} affine(\pi_\theta(o_T))\Delta t - \Delta\theta \|^2 \right]
$$

因此，动作中需要记录的值有：
$$\mathcal{D}_{act}=\{\Delta\theta^E\}_{t-L+1}^{t}$$
关节空间策略输出的动作并不是直接的 $\Delta\theta$，而是归一化后的 $a\in[-1,1]$。从 $a$ 到 $\Delta\theta$，需要经过和各自关节上下限有关的仿射变换。不过可访问动作项 `RelativeJointPositionAction` 的 `processed_actions` 直接获得
<br>于是，便构成了标准的观察-动作对：
$$\mathcal{D} = (\mathcal{D}_O, \mathcal{D}_A) = (\{\theta,\mathcal{V},p_{obj},q_{obj},X_{goal},\Delta q,a_{\theta},a_{\mathcal{V}},J_b\}, \{\Delta\theta\})_{t-L+1}^{t}$$

### 个人迷惑 & 易踩坑点
1. $a_{\mathcal{V}}$ 和 $\mathcal{V_{b'}}$ 的收集
<br>因为 se3-student 策略的观察需要用到 $a_{\mathcal{V}}$,$\mathcal{V_{b'}}$，但 joint-teacher 策略 rollout 无法直接获取 $a_{\mathcal{V}}$,$\mathcal{V_b}$，如何获取是一个问题。
<br>目前的想法是通过运动学关系转换获取：
<br>对于实际末端指尖旋量 $\mathcal{V_{b'}}$，有 $\mathcal{V_{b'}}=Ad_{T_{b'b}}J_b\dot{\theta} $，这里的 $\dot{\theta}$ 可通过 IsaacLab API 获取
<br>对于上一步动作 $a_{\mathcal{V}}$，有 $\text{InverseAffine}(J_b\frac{\Delta\theta}{\Delta t})$，这里的 $\Delta\theta$ 和 $J_b$ 都是上一步关节策略动作的。不过这里用的是实际 $J_b$，而非 $J_{dls}$，主要是感觉 $J_{dls}^+$ 到 $J_{dls}$ 不好求
2. 历史步，hdf5格式问题
<br>如果策略的观察和历史步有关，是否是单个时间步保存 $o_t$，还是 $\{o_{t-L+1},...,o_{t-1},o_t\}$ 作为每一帧？个人倾向单时间步每帧即可，然后在策略里根据历史步数进行拼接。不知道hdf5格式是否支持这种存储方式？
3. 关节索引问题
<br>即`joint_ids`，这个在关节空间和se3空间都要对齐，保证统一的顺序

### 在线

# Multimodal Learning

## 多模态学习
se3在语义上，如果融入了视觉或触觉方面想信息，可能表现的会更好，但如何融合这些信息、构建训练管线是一个挑战